In [ ]:
!pip install feedparser

In [ ]:
# === 환경설정 & 공통 상수 ===
import requests, time, re, json
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path

# 네이버가 빈 페이지/리다이렉트 주는 걸 막기 위한 헤더
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:128.0) Gecko/20100101 Firefox/128.0",
    "Accept-Language": "ko-KR,ko;q=0.9,en;q=0.8",
    "Referer": "https://www.naver.com/"
}

# 프로젝트 경로(노트북 기준)
RAW = Path("../data/raw")
PROC = Path("../data/processed")
RAW.mkdir(parents=True, exist_ok=True)
PROC.mkdir(parents=True, exist_ok=True)

print("✅ 환경 준비 완료")


In [ ]:
# === 기사 URL 패턴 & 유틸 ===
ARTICLE_PATTERNS = [
    re.compile(r"https?://n\.news\.naver\.com/article/\d+/\d+"),
    re.compile(r"https?://n\.news\.naver\.com/mnews/article/\d+/\d+"),
    re.compile(r"https?://news\.naver\.com/main/read\.naver.*[?&]oid=\d+.*[?&]aid=\d+"),
    re.compile(r"https?://mnews\.naver\.com/article/\d+/\d+"),
]

def is_article_url(href: str) -> bool:
    if not href:
        return False
    return any(p.search(href) for p in ARTICLE_PATTERNS)

def extract_oid_aid(url: str):
    # n.news (PC/모바일)
    m1 = re.search(r"n\.news\.naver\.com/(?:mnews/)?article/(\d+)/(\d+)", url)
    if m1:
        return m1.group(1), m1.group(2)
    # news.naver.com read.naver
    m2_oid = re.search(r"[?&]oid=(\d+)", url)
    m2_aid = re.search(r"[?&]aid=(\d+)", url)
    if m2_oid and m2_aid:
        return m2_oid.group(1), m2_aid.group(1)
    return None, None

def resolve_article_title(url, timeout=10):
    """기사 페이지에 직접 접속해 제목(og:title 우선)을 가져온다."""
    try:
        r = requests.get(url, headers=HEADERS, timeout=timeout, allow_redirects=True)
        if r.status_code != 200:
            return None
        soup = BeautifulSoup(r.text, "html.parser")
        og = soup.select_one('meta[property="og:title"]')
        if og and og.get("content"):
            return og["content"].strip()
        if soup.title and soup.title.string:
            return soup.title.string.strip()
    except Exception:
        return None
    return None

print("✅ 유틸 로드")


In [ ]:
# === 뉴스 링크 수집 (검색 결과 HTML → 기사 URL) ===
def get_news_links_html(query, pages=15, sleep_sec=0.25, issue_keywords=None):
    base = "https://search.naver.com/search.naver"
    rows = []

    for start_idx in range(1, pages*10, 10):  # 1, 11, 21, ...
        params = {"where": "news", "query": query, "start": start_idx}
        res = requests.get(base, headers=HEADERS, params=params, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")

        # 대표 선택자 → 없으면 전체 a[href]에서 기사 패턴만
        anchors = soup.select("a.title_link")
        if not anchors:
            anchors = soup.select("a.news_tit")
        if not anchors:
            anchors = [a for a in soup.find_all("a", href=True)]

        kept = 0
        for a in anchors:
            href = a.get("href", "")
            if not is_article_url(href):
                continue
            title = a.get_text(strip=True) or "네이버뉴스"

            # (선택) 제목 키워드 필터
            if issue_keywords:
                t = title.lower()
                if not any(k.lower() in t for k in issue_keywords):
                    continue

            rows.append({"title": title, "url": href, "query": query})
            kept += 1

        print(f"page {start_idx}: kept {kept}")
        if kept == 0 and start_idx == 1:
            break  # 첫 페이지부터 0이면 중단

        time.sleep(sleep_sec)

    df = pd.DataFrame(rows).drop_duplicates(subset=["url"]).reset_index(drop=True)
    return df

print("✅ 링크 수집 함수 준비")


In [ ]:
#롬앤

In [ ]:
# === 실행 파라미터 (브랜드별로 바꿔 쓰기) ===
brand_code = "romand"  # 저장 파일명 등에 사용
brand_query = "롬앤 구순염"   # 검색 쿼리(넓게)
issue_keywords = None 


queries = ["롬앤", "롬앤 틴트", "롬앤 논란", "롬앤 입술", "롬앤 알레르기"]

print("✅ 파라미터:", brand_code, brand_query, issue_keywords)


In [ ]:
# 1) 링크 수집 (넓게)
html_df = get_news_links_html(brand_query, pages=15, issue_keywords=issue_keywords)
print("수집 기사 수:", len(html_df))
display(html_df.head(3))

# 2) 기사 페이지에서 실제 제목(og:title) 해석
if len(html_df):
    html_df["title_resolved"] = html_df["url"].apply(resolve_article_title)
    # 너무 빠른 요청 방지
    time.sleep(0.3)

# 3) 제목 기반 이슈 필터 (필요 시 조정)
issue_kw = ["구순염","염증","알레르기","입술","자극","논란","부작용","사과","보상","기만","해명","문의","CS"]
pat = "|".join(issue_kw)
filtered = (html_df
            .dropna(subset=["title_resolved"])
            .loc[html_df["title_resolved"].str.contains(pat, case=False, na=False)]
            .drop_duplicates(subset=["url"])
            .reset_index(drop=True))

print("이슈 필터 후:", len(filtered))
display(filtered.head(5))

In [ ]:
# === 댓글 API 수집 ===
def get_naver_comments(oid, aid, max_pages=20, sleep_sec=0.2):
    """
    기사 한 개의 댓글을 수집한다. (최대 max_pages, 페이지당 20개)
    반환: 리스트[ {user, contents, regTime}, ... ]
    """
    out = []
    for page in range(1, max_pages + 1):
        api = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json"
        params = {
            "ticket": "news",
            "templateId": "default_news",
            "pool": "cbox5",
            "lang": "ko",
            "country": "KR",
            "objectId": f"news{oid},{aid}",
            "pageSize": 20,
            "pageType": 1,
            "page": page,
            "sort": "FAVORITE"
        }
        try:
            res = requests.get(api, headers=HEADERS, params=params, timeout=10)
        except Exception as e:
            print(f"  comments req err: {e}")
            break

        txt = res.text.strip()
        m = re.search(r"\((\{.*\})\)$", txt)  # JSONP → JSON
        if not m:
            break
        data = json.loads(m.group(1))
        lst = data.get("result", {}).get("commentList", [])
        if not lst:
            break

        for c in lst:
            out.append({
                "user": c.get("userNameMasked"),
                "contents": c.get("contents"),
                "regTime": c.get("regTime")
            })
        time.sleep(sleep_sec)
    return out

def collect_comments_for_links(df_links, brand_code, max_articles=15, max_comment_pages=10):
    """
    링크 DF → oid/aid 추출 → 댓글 수집 → CSV 저장
    """
    rows = []
    target = df_links.head(max_articles)
    for i, r in target.iterrows():
        oid, aid = extract_oid_aid(r["url"])
        if not oid:
            continue
        print(f"[{i+1}/{len(target)}] oid={oid}, aid={aid}")
        cmts = get_naver_comments(oid, aid, max_pages=max_comment_pages)
        for c in cmts:
            rows.append({
                "brand": brand_code,
                "title": r.get("title_resolved") or r.get("title"),
                "url": r["url"],
                "contents": c["contents"],
                "regTime": c["regTime"]
            })
        time.sleep(0.2)

    out = pd.DataFrame(rows)
    out_path = RAW / f"comments_{brand_code}.csv"
    out.to_csv(out_path, index=False, encoding="utf-8-sig")
    print("✅ 저장:", out_path, "rows:", len(out))
    return out


In [ ]:
# 댓글 대상 링크 선택: 필터 결과가 있으면 그걸, 없으면 원본 중 상위 N개
links_for_comments = filtered if len(filtered) else html_df

if len(links_for_comments) == 0:
    print("❗ 기사 링크가 없습니다. query/필터를 완화하거나 pages를 늘려 다시 시도하세요.")
else:
    comments_df = collect_comments_for_links(links_for_comments, brand_code,
                                             max_articles=10, max_comment_pages=10)
    display(comments_df.head(5))


In [ ]:
#클리오

In [ ]:
# === 실행 파라미터 (브랜드별로 바꿔 쓰기) ===
brand_code = "clio"  # 저장 파일명 등에 사용
brand_query = "클리오 후기"   # 검색 쿼리(넓게)
issue_keywords = None 


queries = ["클리오", "클리오 쿠션", "클리오 논란", "클리오 입술", "클리오 알레르기"]

print("✅ 파라미터:", brand_code, brand_query, issue_keywords)


In [ ]:
# 1) 링크 수집 (넓게)
html_df = get_news_links_html(brand_query, pages=15, issue_keywords=issue_keywords)
print("수집 기사 수:", len(html_df))
display(html_df.head(3))

# 2) 기사 페이지에서 실제 제목(og:title) 해석
if len(html_df):
    html_df["title_resolved"] = html_df["url"].apply(resolve_article_title)
    # 너무 빠른 요청 방지
    time.sleep(0.3)

# 3) 제목 기반 이슈 필터 (필요 시 조정)
issue_kw = ["클리오","각질 부각","피부병","피부","자극","논란","부작용","보상","기만","해명","문의","CS"]
pat = "|".join(issue_kw)
filtered = (html_df
            .dropna(subset=["title_resolved"])
            .loc[html_df["title_resolved"].str.contains(pat, case=False, na=False)]
            .drop_duplicates(subset=["url"])
            .reset_index(drop=True))

print("이슈 필터 후:", len(filtered))
display(filtered.head(5))

In [ ]:
# === 댓글 API 수집 ===
def get_naver_comments(oid, aid, max_pages=20, sleep_sec=0.2):
    """
    기사 한 개의 댓글을 수집한다. (최대 max_pages, 페이지당 20개)
    반환: 리스트[ {user, contents, regTime}, ... ]
    """
    out = []
    for page in range(1, max_pages + 1):
        api = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json"
        params = {
            "ticket": "news",
            "templateId": "default_news",
            "pool": "cbox5",
            "lang": "ko",
            "country": "KR",
            "objectId": f"news{oid},{aid}",
            "pageSize": 20,
            "pageType": 1,
            "page": page,
            "sort": "FAVORITE"
        }
        try:
            res = requests.get(api, headers=HEADERS, params=params, timeout=10)
        except Exception as e:
            print(f"  comments req err: {e}")
            break

        txt = res.text.strip()
        m = re.search(r"\((\{.*\})\)$", txt)  # JSONP → JSON
        if not m:
            break
        data = json.loads(m.group(1))
        lst = data.get("result", {}).get("commentList", [])
        if not lst:
            break

        for c in lst:
            out.append({
                "user": c.get("userNameMasked"),
                "contents": c.get("contents"),
                "regTime": c.get("regTime")
            })
        time.sleep(sleep_sec)
    return out

def collect_comments_for_links(df_links, brand_code, max_articles=15, max_comment_pages=10):
    """
    링크 DF → oid/aid 추출 → 댓글 수집 → CSV 저장
    """
    rows = []
    target = df_links.head(max_articles)
    for i, r in target.iterrows():
        oid, aid = extract_oid_aid(r["url"])
        if not oid:
            continue
        print(f"[{i+1}/{len(target)}] oid={oid}, aid={aid}")
        cmts = get_naver_comments(oid, aid, max_pages=max_comment_pages)
        for c in cmts:
            rows.append({
                "brand": brand_code,
                "title": r.get("title_resolved") or r.get("title"),
                "url": r["url"],
                "contents": c["contents"],
                "regTime": c["regTime"]
            })
        time.sleep(0.2)

    out = pd.DataFrame(rows)
    out_path = RAW / f"comments_{brand_code}.csv"
    out.to_csv(out_path, index=False, encoding="utf-8-sig")
    print("✅ 저장:", out_path, "rows:", len(out))
    return out


In [ ]:
# 댓글 대상 링크 선택: 필터 결과가 있으면 그걸, 없으면 원본 중 상위 N개
links_for_comments = filtered if len(filtered) else html_df

if len(links_for_comments) == 0:
    print("❗ 기사 링크가 없습니다. query/필터를 완화하거나 pages를 늘려 다시 시도하세요.")
else:
    comments_df = collect_comments_for_links(links_for_comments, brand_code,
                                             max_articles=10, max_comment_pages=10)
    display(comments_df.head(5))


In [ ]:
#에뛰드

In [ ]:
# === 실행 파라미터 (브랜드별로 바꿔 쓰기) ===
brand_code = "etude"  # 저장 파일명 등에 사용
brand_query = "에뛰드 논란"   # 검색 쿼리(넓게)
issue_keywords = None 


queries = ["에뛰드", "에뛰드 판매중지", "중금속", "구순염", "알레르기"]

print("✅ 파라미터:", brand_code, brand_query, issue_keywords)


In [ ]:
# 1) 링크 수집 (넓게)
html_df = get_news_links_html(brand_query, pages=15, issue_keywords=issue_keywords)
print("수집 기사 수:", len(html_df))
display(html_df.head(3))

# 2) 기사 페이지에서 실제 제목(og:title) 해석
if len(html_df):
    html_df["title_resolved"] = html_df["url"].apply(resolve_article_title)
    # 너무 빠른 요청 방지
    time.sleep(0.3)

# 3) 제목 기반 이슈 필터 (필요 시 조정)
issue_kw = ["에뛰드","중금속","피부병","논란","부작용","보상","기만","해명","자극","성분"]
pat = "|".join(issue_kw)
filtered = (html_df
            .dropna(subset=["title_resolved"])
            .loc[html_df["title_resolved"].str.contains(pat, case=False, na=False)]
            .drop_duplicates(subset=["url"])
            .reset_index(drop=True))

print("이슈 필터 후:", len(filtered))
display(filtered.head(5))

In [ ]:
# === 댓글 API 수집 ===
def get_naver_comments(oid, aid, max_pages=20, sleep_sec=0.2):
    """
    기사 한 개의 댓글을 수집한다. (최대 max_pages, 페이지당 20개)
    반환: 리스트[ {user, contents, regTime}, ... ]
    """
    out = []
    for page in range(1, max_pages + 1):
        api = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json"
        params = {
            "ticket": "news",
            "templateId": "default_news",
            "pool": "cbox5",
            "lang": "ko",
            "country": "KR",
            "objectId": f"news{oid},{aid}",
            "pageSize": 20,
            "pageType": 1,
            "page": page,
            "sort": "FAVORITE"
        }
        try:
            res = requests.get(api, headers=HEADERS, params=params, timeout=10)
        except Exception as e:
            print(f"  comments req err: {e}")
            break

        txt = res.text.strip()
        m = re.search(r"\((\{.*\})\)$", txt)  # JSONP → JSON
        if not m:
            break
        data = json.loads(m.group(1))
        lst = data.get("result", {}).get("commentList", [])
        if not lst:
            break

        for c in lst:
            out.append({
                "user": c.get("userNameMasked"),
                "contents": c.get("contents"),
                "regTime": c.get("regTime")
            })
        time.sleep(sleep_sec)
    return out

def collect_comments_for_links(df_links, brand_code, max_articles=15, max_comment_pages=10):
    """
    링크 DF → oid/aid 추출 → 댓글 수집 → CSV 저장
    """
    rows = []
    target = df_links.head(max_articles)
    for i, r in target.iterrows():
        oid, aid = extract_oid_aid(r["url"])
        if not oid:
            continue
        print(f"[{i+1}/{len(target)}] oid={oid}, aid={aid}")
        cmts = get_naver_comments(oid, aid, max_pages=max_comment_pages)
        for c in cmts:
            rows.append({
                "brand": brand_code,
                "title": r.get("title_resolved") or r.get("title"),
                "url": r["url"],
                "contents": c["contents"],
                "regTime": c["regTime"]
            })
        time.sleep(0.2)

    out = pd.DataFrame(rows)
    out_path = RAW / f"comments_{brand_code}.csv"
    out.to_csv(out_path, index=False, encoding="utf-8-sig")
    print("✅ 저장:", out_path, "rows:", len(out))
    return out


In [ ]:
# 댓글 대상 링크 선택: 필터 결과가 있으면 그걸, 없으면 원본 중 상위 N개
links_for_comments = filtered if len(filtered) else html_df

if len(links_for_comments) == 0:
    print("❗ 기사 링크가 없습니다. query/필터를 완화하거나 pages를 늘려 다시 시도하세요.")
else:
    comments_df = collect_comments_for_links(links_for_comments, brand_code,
                                             max_articles=10, max_comment_pages=10)
    display(comments_df.head(5))


In [ ]:
#이니스프리 

In [ ]:
# === 실행 파라미터 (브랜드별로 바꿔 쓰기) ===
brand_code = "innisfree"  # 저장 파일명 등에 사용
brand_query = "이니스프리 논란"   # 검색 쿼리(넓게)
issue_keywords = None 


queries = ["이니스프리", "종이공병", "피부병", "판매중지", "알레르기"]

print("✅ 파라미터:", brand_code, brand_query, issue_keywords)


In [ ]:
# 1) 링크 수집 (넓게)
html_df = get_news_links_html(brand_query, pages=15, issue_keywords=issue_keywords)
print("수집 기사 수:", len(html_df))
display(html_df.head(3))

# 2) 기사 페이지에서 실제 제목(og:title) 해석
if len(html_df):
    html_df["title_resolved"] = html_df["url"].apply(resolve_article_title)
    # 너무 빠른 요청 방지
    time.sleep(0.3)

# 3) 제목 기반 이슈 필터 (필요 시 조정)
issue_kw = ["이니스프리","종이병","재활용","논란","부작용","보상","기만","해명","자극","성분"]
pat = "|".join(issue_kw)
filtered = (html_df
            .dropna(subset=["title_resolved"])
            .loc[html_df["title_resolved"].str.contains(pat, case=False, na=False)]
            .drop_duplicates(subset=["url"])
            .reset_index(drop=True))

print("이슈 필터 후:", len(filtered))
display(filtered.head(5))

In [ ]:
# === 댓글 API 수집 ===
def get_naver_comments(oid, aid, max_pages=20, sleep_sec=0.2):
    """
    기사 한 개의 댓글을 수집한다. (최대 max_pages, 페이지당 20개)
    반환: 리스트[ {user, contents, regTime}, ... ]
    """
    out = []
    for page in range(1, max_pages + 1):
        api = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json"
        params = {
            "ticket": "news",
            "templateId": "default_news",
            "pool": "cbox5",
            "lang": "ko",
            "country": "KR",
            "objectId": f"news{oid},{aid}",
            "pageSize": 20,
            "pageType": 1,
            "page": page,
            "sort": "FAVORITE"
        }
        try:
            res = requests.get(api, headers=HEADERS, params=params, timeout=10)
        except Exception as e:
            print(f"  comments req err: {e}")
            break

        txt = res.text.strip()
        m = re.search(r"\((\{.*\})\)$", txt)  # JSONP → JSON
        if not m:
            break
        data = json.loads(m.group(1))
        lst = data.get("result", {}).get("commentList", [])
        if not lst:
            break

        for c in lst:
            out.append({
                "user": c.get("userNameMasked"),
                "contents": c.get("contents"),
                "regTime": c.get("regTime")
            })
        time.sleep(sleep_sec)
    return out

def collect_comments_for_links(df_links, brand_code, max_articles=15, max_comment_pages=10):
    """
    링크 DF → oid/aid 추출 → 댓글 수집 → CSV 저장
    """
    rows = []
    target = df_links.head(max_articles)
    for i, r in target.iterrows():
        oid, aid = extract_oid_aid(r["url"])
        if not oid:
            continue
        print(f"[{i+1}/{len(target)}] oid={oid}, aid={aid}")
        cmts = get_naver_comments(oid, aid, max_pages=max_comment_pages)
        for c in cmts:
            rows.append({
                "brand": brand_code,
                "title": r.get("title_resolved") or r.get("title"),
                "url": r["url"],
                "contents": c["contents"],
                "regTime": c["regTime"]
            })
        time.sleep(0.2)

    out = pd.DataFrame(rows)
    out_path = RAW / f"comments_{brand_code}.csv"
    out.to_csv(out_path, index=False, encoding="utf-8-sig")
    print("✅ 저장:", out_path, "rows:", len(out))
    return out


In [ ]:
# 댓글 대상 링크 선택: 필터 결과가 있으면 그걸, 없으면 원본 중 상위 N개
links_for_comments = filtered if len(filtered) else html_df

if len(links_for_comments) == 0:
    print("❗ 기사 링크가 없습니다. query/필터를 완화하거나 pages를 늘려 다시 시도하세요.")
else:
    comments_df = collect_comments_for_links(links_for_comments, brand_code,
                                             max_articles=10, max_comment_pages=10)
    display(comments_df.head(5))
